In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv')
df.head(2)

In [ ]:
df['len']= df['full_text'].str.len().astype(int)
df['word'] = df['full_text'].apply(lambda row1: len(row1.split()))
df.head(3)

In [ ]:
print(df['full_text'][0])

In [ ]:
df['score'].unique()

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
sns.countplot(x = 'score', data = df,palette=['#432371',"#FAAE7B"])
plt.show()

In [ ]:
df['Length'] = df['full_text'].apply(len)
df.sample(2)

In [ ]:
sns.histplot(x = 'Length',data = df,edgecolor = 'black',bins = 50)
plt.show()

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Download necessary libraries (nltk.download('stopwords'))
nltk.download('stopwords')

In [ ]:
import string
def nonpunct(mess):
    nonpunc = [x for x in mess if x  not in string.punctuation]
    nonpunc = ''.join(nonpunc).lower()
    return nonpunc
df['full_text'] = df['full_text'].apply(nonpunct)
df['full_text'][0]

In [ ]:
def remove_tags(text):
    cleaned_text = text.replace('\xa0', ' ')
    cleaned_text = re.sub(r"^[a-zA-Z]", "", cleaned_text)
    return cleaned_text
df['full_text'] = df['full_text'].apply(remove_tags)
df['full_text'][0]

In [ ]:
df['full_text_without_stopwords'] = df['full_text'].apply(lambda x: ' '.join([word for word in x.split() 
                                                                              if word not in stopwords.words('english')]))

df['full_text_without_stopwords'][0]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer().fit(df['full_text'])
print(len(cv.vocabulary_))

In [ ]:
print(len(cv.vocabulary_))

In [ ]:
full_text_bow = cv.transform(df['full_text'])
print(full_text_bow)
type(full_text_bow)

In [ ]:
mess1 = df['full_text'][0]
bow1 = cv.transform([mess1])
text_bow = cv.transform(df['full_text'])

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
TT = TfidfTransformer().fit(full_text_bow)
tfidf = TT.transform(bow1)
print(tfidf)

In [ ]:
text_tfidf=  TT.transform(text_bow)
print(text_tfidf.shape)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB(alpha = 0.1).fit(text_tfidf, df['score'])

In [ ]:
prediction = model.predict(text_tfidf)
print(prediction)

In [ ]:
model.score(text_tfidf, df['score'])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(df['score'],prediction))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
model1 = RandomForestClassifier()


In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [4, 8, 12],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [ ]:
random_search = RandomizedSearchCV(model1, param_distributions=param_grid, n_iter=10, cv=5, scoring='accuracy', random_state=42)
random_search.fit(text_tfidf, df['score'])

In [ ]:
prediction1 = random_search.predict(text_tfidf)
print(prediction1)

In [ ]:
random_search.score(text_tfidf, df['score'])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(df['score'],prediction))

In [ ]:
from sklearn.linear_model import LogisticRegression
model2 = LogisticRegression(random_state=0, C=1.0, penalty='l2', solver='lbfgs', max_iter=1000).fit(text_tfidf, df['score'])

In [ ]:
prediction2 = model2.predict(text_tfidf)
print(prediction2)

In [ ]:
model2.score(text_tfidf, df['score'])